# Desafio Churn — Previsão de Evasão de Clientes

**Autor:** Fellipe José

**Resumo:** Notebook funcional que implementa todo o fluxo pedido: carregamento automático do dataset, pré-processamento (com destaque para a lógica de tratamento de nulos e codificação), modelagem com Regressão Logística, avaliação, cross-validation e função de deploy `prever_novo_cliente()`.

---


## 1) Setup e download automático do dataset

O notebook tenta primeiro localizar um arquivo local `Telco-Customer-Churn.csv`. Caso não exista, ele tenta baixar automaticamente de um repositório público (raw GitHub). Se o download falhar, o usuário pode subir manualmente o CSV na mesma pasta do notebook.

**Observação:** mantenha esse notebook e o CSV no mesmo diretório ao enviar para o GitHub para o professor.

In [ ]:
import os
import pandas as pd
from pathlib import Path
from urllib.request import urlretrieve

local_filename = 'Telco-Customer-Churn.csv'
download_url = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'

if not Path(local_filename).exists():
    print(f"Arquivo local '{local_filename}' não encontrado. Tentando baixar de:\n{download_url}")
    try:
        urlretrieve(download_url, local_filename)
        print('Download concluído com sucesso.')
    except Exception as e:
        print('Falha ao baixar automaticamente. Por favor, coloque o arquivo CSV na mesma pasta do notebook.')
        print('Erro:', e)

if Path(local_filename).exists():
    df = pd.read_csv(local_filename)
    print('Dataset carregado:', local_filename)
else:
    print('Nenhum dataset disponível no diretório. Pare o notebook e envie o CSV ou verifique a URL de download.')

Arquivo local 'Telco-Customer-Churn.csv' não encontrado. Tentando baixar de:
https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv
Download concluído com sucesso.
Dataset carregado: Telco-Customer-Churn.csv


## 2) Coleta e Entendimento dos Dados

Mostraremos as 10 primeiras linhas, os tipos de dados e a contagem de valores ausentes por coluna.

In [ ]:
try:
    display(df.head(10))
    print('\nTipos de dados:\n')
    print(df.dtypes)
    print('\nValores ausentes por coluna:\n')
    print(df.isnull().sum())
except NameError:
    print('O dataset não foi carregado. Execute a célula de download / coloque o CSV no diretório.')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No



Tipos de dados:

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

Valores ausentes por coluna:

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract           

## 3) Tratamento / Preparação dos Dados (Foco em Lógica e Algoritmos)

Aqui implementamos a lógica para tratar valores nulos, codificar categóricas via One-Hot Encoding e escalonar variáveis numéricas. Toda a lógica é comentada em cada bloco de código.

In [ ]:
import numpy as np

# Faz uma cópia para segurança
df_proc = df.copy()

# Identificar colunas com nulos usando um loop e tratar de acordo com o tipo
nulos = {}
for col in df_proc.columns:
    n_missing = df_proc[col].isnull().sum()
    if n_missing > 0:
        nulos[col] = n_missing
        if df_proc[col].dtype in [np.float64, np.int64]:
            median_val = df_proc[col].median()
            df_proc[col].fillna(median_val, inplace=True)
            print(f"Coluna '{col}': {n_missing} nulos -> preenchidos com mediana ({median_val})")
        else:
            df_proc[col].fillna('Desconhecido', inplace=True)
            print(f"Coluna '{col}': {n_missing} nulos -> preenchidos com 'Desconhecido'")

if not nulos:
    print('Não foram encontrados valores nulos no dataset original.')
else:
    print('\nResumo dos nulos tratados:', nulos)

Não foram encontrados valores nulos no dataset original.


In [ ]:
cat_cols = df_proc.select_dtypes(include=['object']).columns.tolist()
num_cols = df_proc.select_dtypes(include=['int64','float64']).columns.tolist()

print('Colunas categóricas:', len(cat_cols), 'exemplos ->', cat_cols[:10])
print('Colunas numéricas:', len(num_cols), '->', num_cols)

Colunas categóricas: 18 exemplos -> ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection']
Colunas numéricas: 3 -> ['SeniorCitizen', 'tenure', 'MonthlyCharges']


### 3b) One-Hot Encoding (lógica algorítmica)

One-Hot Encoding transforma cada categoria em uma coluna binária (0/1). Isso evita que o modelo interprete ordens entre categorias (o que ocorreria se usássemos label encoding). Usaremos `OneHotEncoder(handle_unknown='ignore')` do scikit-learn dentro de um `ColumnTransformer`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

# Separar a coluna alvo 'Churn' se existir (ajuste para o dataset IBM: coluna 'Churn')
target_col = 'Churn' if 'Churn' in df_proc.columns else df_proc.columns[-1]
print('Coluna alvo assumida:', target_col)

# Retirar identificadores que não são úteis para modelagem (se existirem)
drop_cols = ['customerID'] if 'customerID' in df_proc.columns else []
if drop_cols:
    df_proc.drop(columns=drop_cols, inplace=True)

X = df_proc.drop(columns=[target_col])
y = df_proc[target_col].apply(lambda x: 1 if str(x).strip().lower() in ['yes','1','true'] else 0)

# Recomputar listas de colunas após possíveis drops
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()

cat_cols, num_cols[:10]

Coluna alvo assumida: Churn


(['gender',
  'Partner',
  'Dependents',
  'PhoneService',
  'MultipleLines',
  'InternetService',
  'OnlineSecurity',
  'OnlineBackup',
  'DeviceProtection',
  'TechSupport',
  'StreamingTV',
  'StreamingMovies',
  'Contract',
  'PaperlessBilling',
  'PaymentMethod',
  'TotalCharges'],
 ['SeniorCitizen', 'tenure', 'MonthlyCharges'])

### 3c) Normalização/Escalonamento (MinMaxScaler)

Aplicaremos `MinMaxScaler` às colunas numéricas para deixar os features em escala [0,1], o que ajuda a convergência da Regressão Logística.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

# Pré-processador com OneHot para categóricas e MinMaxScaler para numéricas
preprocessor = ColumnTransformer(transformers=[
    ('num', MinMaxScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
], remainder='drop')

# Pipeline completo
pipeline = Pipeline(steps=[
    ('preproc', preprocessor),
    ('clf', LogisticRegression(solver='liblinear', max_iter=1000))
])

print('Pipeline pronto. Pré-processador configurado com OneHotEncoder e MinMaxScaler.')

Pipeline pronto. Pré-processador configurado com OneHotEncoder e MinMaxScaler.


## 4) Análise Exploratória (proporção de Churn)

Calcule a proporção de clientes que deram churn e comente o impacto do desbalanceamento.

In [ ]:
from sklearn.model_selection import train_test_split
# Proporção de churn
try:
    prop = y.value_counts(normalize=True)
    print('Proporção das classes (0 = Não Churn, 1 = Churn):\n', prop)
except NameError:
    print('Variável alvo não definida.')

# Dividir dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print('\nShapes -> X_train:', X_train.shape, 'X_test:', X_test.shape)

Proporção das classes (0 = Não Churn, 1 = Churn):
 Churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64

Shapes -> X_train: (5634, 19) X_test: (1409, 19)


## 5) Modelagem — Regressão Logística

Treinaremos a Regressão Logística e explicaremos a função sigmoide no texto a seguir.

### Função sigmoide (explicação rápida)

A função sigmoide transforma a combinação linear dos pesos e features z = w^T x + b em uma probabilidade:

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

Assim, valores muito negativos vão para 0, valores muito positivos vão para 1. A regressão logística usa isso para modelar a probabilidade de classe positiva.

In [ ]:
pipeline.fit(X_train, y_train)
print('Treinamento concluído.')

Treinamento concluído.


## 6) Avaliação do Modelo

Calcularemos Acurácia, Precisão, Recall e F1-Score. Para o problema de retenção, justificamos priorizar Recall (capturar o máximo de clientes que saem).

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:,1]

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Acurácia: {:.4f}'.format(acc))
print('Precisão: {:.4f}'.format(prec))
print('Recall: {:.4f}'.format(rec))
print('F1-score: {:.4f}'.format(f1))
print('\nClassification report:\n')
print(classification_report(y_test, y_pred))
print('\nMatriz de confusão:\n', confusion_matrix(y_test, y_pred))

Acurácia: 0.7942
Precisão: 0.6304
Recall: 0.5428
F1-score: 0.5833

Classification report:

              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1035
           1       0.63      0.54      0.58       374

    accuracy                           0.79      1409
   macro avg       0.74      0.71      0.72      1409
weighted avg       0.79      0.79      0.79      1409


Matriz de confusão:
 [[916 119]
 [171 203]]


### 6b) Otimização: Validação Cruzada

Aplicaremos cross-validation para obter uma avaliação mais robusta do desempenho do modelo.

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(pipeline, X, y, cv=cv, scoring='recall')
print('Recall por fold:', scores)
print('Recall médio (CV): {:.4f} +/- {:.4f}'.format(scores.mean(), scores.std()))

Recall por fold: [0.56951872 0.57486631 0.57486631 0.49597855 0.51604278]
Recall médio (CV): 0.5463 +/- 0.0335


## 7) Deploy simulado — função `prever_novo_cliente(dados_cliente)`

A função recebe um dicionário com as features do cliente (mesmos nomes das colunas do dataset original), aplica o pré-processamento e retorna probabilidade e decisão (Churn/Não Churn).

In [ ]:
def prever_novo_cliente(dados_cliente: dict):
    # Converter para DataFrame de uma linha
    df_new = pd.DataFrame([dados_cliente])

    # Garantir que todas as colunas esperadas existam (preencher ausentes com valores neutros)
    for c in X.columns:
        if c not in df_new.columns:
            if c in num_cols:
                df_new[c] = 0
            else:
                df_new[c] = 'Desconhecido'

    df_new = df_new[X.columns]
    proba = pipeline.predict_proba(df_new)[:,1][0]
    pred = pipeline.predict(df_new)[0]
    decision = 'Churn' if pred == 1 else 'Não Churn'
    return {'probabilidade_churn': float(proba), 'decisao': decision}

# Exemplo de uso (substitua valores conforme o dataset)
example = {}
for c in X.columns:
    if c in num_cols:
        try:
            example[c] = X[c].median()
        except Exception:
            example[c] = 0
    else:
        example[c] = X[c].iloc[0] if len(X)>0 else 'Desconhecido'

print('Exemplo de predição para um cliente fictício:')
print(prever_novo_cliente(example))

Exemplo de predição para um cliente fictício:
{'probabilidade_churn': 0.22353997077051926, 'decisao': 'Não Churn'}
